##PyCitySchools Analysis


# My conclusions based on the analysis of the data below are:

Charter schools outperformed district schools by 17 points.  The overall passing rate for charter schools is 95.1% versus 73.7% for district schools.

Not unsurprisingly, larger schools (in excess of 2,000 students) performed poorly versus small and medium sized schools.

Paying the least amount per student, per budget results, actually generates the second best overall passing rate: 95 versus 95.1. Since the difference is de minimis the smart money is on finding a way to reduce the overal budgets by paying only $585 per student.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# Import csv files
schools = 'Resources/schools_complete.csv'
students = 'Resources/students_complete.csv'

In [3]:
# Open and read school and student data
# format decimals
pd.options.display.float_format = '{:,.2f}'.format
schools_df = pd.read_csv(schools)
students_df = pd.read_csv(students)
#students_df.head()

In [4]:
# Rename columns
schools = schools_df.rename(columns={"school_name": "school"})
students = students_df.rename(columns={"student_name": "student", "school_name": "school"})

In [5]:
# Change budget and size data into integers
schools ["budget"] = pd.to_numeric(schools ["budget"])
schools ["size"] = pd.to_numeric(schools ["size"])

In [6]:
# Combine the data into a single dataset.  
pycityschool_df = pd.merge(schools,students,how="left", on=["school"])
#pycityschool_df.head()["budget"])
pycityschool_df["reading_score"] = pd.to_numeric(pycityschool_df["reading_score"])
pycityschool_df["math_score"] = pd.to_numeric(pycityschool_df["math_score"])
pycityschool_df["size"] = pd.to_numeric(pycityschool_df["size"])
#pycityschool_df.head()

##High Level Snapshot (as table) of PyCitySchools' District Key Metrics

In [7]:
# Calculate the total number of schools, students, and budget from the schools' dataframe
total_schools = schools_df["School ID"].count()
total_students = students_df["Student ID"].count()
total_budget = schools_df["budget"].sum()

In [8]:
# Calculate Average Math Score,Reading Score, and the Percent Passing in Math, Reading, and Overall (in both subjects)
average_math = students_df['math_score'].mean()
average_reading = students_df['reading_score'].mean()
pass_math = (students_df['math_score']>=70).sum()/total_students
pass_reading = (students_df['reading_score']>=70).sum()/total_students
pass_overall = (pass_math+pass_reading)/2

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [9]:
# Display District Summary with cleaner formatting, floats to 2 decimals, and budget using $ sign.
district_summary_df = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Budget":[total_budget],
    "Total Students":[total_students],
    "Avg Math Score":[average_math],
    "Avg Reading Score":[average_reading],
    "Percentage Passing Math":[pass_math*100],
    "Percentage Passing Reading":[pass_reading*100],
    "Percentage Passing Overall":[pass_overall*100]})
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,.2f}'.format)
district_summary_df.head()

,Total Schools,Total Budget,Total Students,Avg Math Score,Avg Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Overall
0,15,"$24,649,428.00",39170,78.99,81.88,74.98,85.81,80.39


##School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [10]:
pycityschool_df.head()

,School ID,school,type,size,budget,Student ID,student,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [11]:
schools.head()

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [12]:
# for the summary key metrics table, group pycityschool_df data by school name 

grouped_school = pycityschool_df.set_index('school').groupby(['school'])

school_type = schools.set_index('school')['type']

total_student = grouped_school ['Student ID'].count()

school_budget = schools.set_index('school')['budget']

student_budget = schools.set_index('school')['budget']/schools.set_index('school')['size']

avg_math = grouped_school ['math_score'].mean()
avg_read = grouped_school ['reading_score'].mean()

# percent passing
pass_math = pycityschool_df [pycityschool_df ['math_score'] >=70].groupby('school')['Student ID'].count()/total_student*100

pass_read = pycityschool_df [pycityschool_df ['reading_score'] >=70].groupby('school')['Student ID'].count()/total_student*100

overall_passing = (pass_math + pass_read) / 2

school_summary_df = pd.DataFrame({
    "School Type": school_type,
    "Total Students": total_student,
    "Total School Budget": school_budget,
    "Per Student Budget": student_budget,    
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall_passing
})
# format table
school_summary_df["Total School Budget"] = school_summary_df["Total Students"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] =school_summary_df["Per Student Budget"].map("${:,.2f}".format)
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$4,976.00",$628.00,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,1858,"$1,858.00",$582.00,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,2949,"$2,949.00",$639.00,76.71,81.16,65.99,80.74,73.36
Ford High School,District,2739,"$2,739.00",$644.00,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,1468,"$1,468.00",$625.00,83.35,83.82,93.39,97.14,95.27
Hernandez High School,District,4635,"$4,635.00",$652.00,77.29,80.93,66.75,80.86,73.81
Holden High School,Charter,427,$427.00,$581.00,83.80,83.81,92.51,96.25,94.38
Huang High School,District,2917,"$2,917.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$4,761.00",$650.00,77.07,80.97,66.06,81.22,73.64
Pena High School,Charter,962,$962.00,$609.00,83.84,84.04,94.59,95.95,95.27


# Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [13]:
# sort df by passing rate and print the top 5 rows

Top_schools = school_summary_df.sort_values(['Overall Passing Rate'], ascending=[False])
Top_schools .head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,1858,"$1,858.00",$582.00,83.06,83.98,94.13,97.04,95.59
Thomas High School,Charter,1635,"$1,635.00",$638.00,83.42,83.85,93.27,97.31,95.29
Pena High School,Charter,962,$962.00,$609.00,83.84,84.04,94.59,95.95,95.27
Griffin High School,Charter,1468,"$1,468.00",$625.00,83.35,83.82,93.39,97.14,95.27
Wilson High School,Charter,2283,"$2,283.00",$578.00,83.27,83.99,93.87,96.54,95.20


# Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [14]:
# print last 5 row from sorted df, and re-sort ascending by passing rate
Bottom_schools = school_summary_df.sort_values(['Overall Passing Rate'], ascending=[True])
Bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$3,999.00",$637.00,76.84,80.74,66.37,80.22,73.29
Figueroa High School,District,2949,"$2,949.00",$639.00,76.71,81.16,65.99,80.74,73.36
Huang High School,District,2917,"$2,917.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$4,761.00",$650.00,77.07,80.97,66.06,81.22,73.64
Ford High School,District,2739,"$2,739.00",$644.00,77.10,80.75,68.31,79.30,73.80


# Math Scores By Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [15]:
#pycityschool_df

In [17]:
# Create a table that lists the average Math Score for students of each grade level
#(9th, 10th, 11th, 12th) at each school.

ninth = pycityschool_df[(pycityschool_df["grade"] == "9th")]
tenth = pycityschool_df[(pycityschool_df["grade"] == "10th")]
eleventh = pycityschool_df[(pycityschool_df["grade"] == "11th")]
twelfth = pycityschool_df[(pycityschool_df["grade"] == "12th")]

ninthscore = ninth.groupby("school")["math_score"].mean()
tenthscore = tenth.groupby("school")["math_score"].mean()
eleventhscore = eleventh.groupby("school")["math_score"].mean()
twelfthscore = twelfth.groupby("school")["math_score"].mean()

mathscores=pd.DataFrame({
    "9th":ninthscore,
    "10th":tenthscore,
    "11th":eleventhscore,
    "12th":twelfthscore
})
mathscores

,9th,10th,11th,12th
school,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


# Reading Scores By Grade

* Perform the same operations as above for reading scores

In [ ]:
#pycityschool_df

In [18]:
# Create a table that lists the average Reading Score for students of each grade level
#(9th, 10th, 11th, 12th) at each school.

nine = pycityschool_df[(pycityschool_df["grade"] == "9th")]
ten = pycityschool_df[(pycityschool_df["grade"] == "10th")]
eleven = pycityschool_df[(pycityschool_df["grade"] == "11th")]
twelve = pycityschool_df[(pycityschool_df["grade"] == "12th")]

nineRscore = ninth.groupby("school")["reading_score"].mean()
tenRscore = tenth.groupby("school")["reading_score"].mean()
elevenRscore = eleventh.groupby("school")["reading_score"].mean()
twelveRscore = twelfth.groupby("school")["reading_score"].mean()

readingscore=pd.DataFrame({
    "9th":nineRscore,
    "10th":tenRscore,
    "11th":elevenRscore,
    "12th":twelveRscore})
readingscore

,9th,10th,11th,12th
school,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


# Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
##school_summary_df

In [23]:
school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$4,976.00",$628.00,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,1858,"$1,858.00",$582.00,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,2949,"$2,949.00",$639.00,76.71,81.16,65.99,80.74,73.36
Ford High School,District,2739,"$2,739.00",$644.00,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,1468,"$1,468.00",$625.00,83.35,83.82,93.39,97.14,95.27


In [24]:
#Create a table that breaks down school performances based on average Spending Ranges (Per Student)
# use a copy of school_summary_df
school_spending = school_summary_df.copy()
school_spending
spending_bins = [0, 585, 615, 645, 675]
group_names = ["< $585", "$585-615","$615-645","$645-675"]

#student_budget = schools.set_index('school')['budget']/schools.set_index('school')['size']
school_spending['Spending Ranges (Per Student)']= pd.cut(student_budget, spending_bins, labels = group_names)
group_math_score = school_spending.groupby(["Spending Ranges (Per Student)"]).mean()['Average Math Score']
group_reading_score = school_spending.groupby(["Spending Ranges (Per Student)"]).mean()['Average Reading Score']
group_passing_math =  school_spending.groupby(["Spending Ranges (Per Student)"]).mean()['% Passing Math']
group_passing_reading =  school_spending.groupby(["Spending Ranges (Per Student)"]).mean()['% Passing Reading']
group_overall_rate =  (group_passing_math + group_passing_reading) / 2

school_spending = pd.DataFrame({"Average Math Score":group_math_score, 
                               "Average Reading Score":group_reading_score,
                               "% Passing Math":group_passing_math,
                               "% Passing Reading":group_passing_reading,
                               "Overall Passing Rate":group_overall_rate}) 
school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
< $585,83.46,83.93,93.46,96.61,95.04
$585-615,83.60,83.89,94.23,95.90,95.07
$615-645,79.08,81.89,75.67,86.11,80.89
$645-675,77.00,81.03,66.16,81.13,73.65


# Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
#school_summary_df

In [25]:
# Create a table that breaks down school performances based on school size.
# use a copy of school_summary_df
school_size_summary = school_summary_df.copy()

size_bins = [0, 1000, 2000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_size_summary['School Size'] = pd.cut(total_student, size_bins, labels = size_names)

size_math_score = school_size_summary.groupby(['School Size']).mean()['Average Math Score']
size_reading_score = school_size_summary.groupby(['School Size']).mean()['Average Reading Score']
size_passing_math =  school_size_summary.groupby(['School Size']).mean()['% Passing Math']
size_passing_reading =  school_size_summary.groupby(['School Size']).mean()['% Passing Reading']
size_overall_rate =  (size_passing_math + size_passing_reading) / 2

school_size_summary = pd.DataFrame({"Average Math Score":size_math_score, 
                               "Average Reading Score":size_reading_score,
                               "% Passing Math":size_passing_math,
                               "% Passing Reading":size_passing_reading,
                               "Overall Passing Rate":size_overall_rate}) 
school_size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.86,93.60,96.79,95.20
Large (2000-5000),77.75,81.34,69.96,82.77,76.36


# Scores by School Type

* Perform the same operations as above, based on school type

In [26]:
# Create a table that breaks down school performances based on school type
# use a copy of school_summary_df
school_type_summary = school_summary_df.copy()

type_math = school_type_summary.groupby("School Type")["Average Math Score"].mean()
type_reading = school_type_summary.groupby("School Type")["Average Reading Score"].mean()
type_passing_math = school_type_summary.groupby("School Type")["% Passing Math"].mean() 
type_passing_reading = school_type_summary.groupby("School Type")["% Passing Reading"].mean()
type_overall_rate = (type_passing_math + type_passing_reading)/2

school_type_summary = pd.DataFrame({
    "Average Math Score":type_math,
    "Average Reading Score":type_reading,
    "% Passing Math":type_passing_math,
    "% Passing Reading":type_passing_reading,
    "Overall Passing Rate":type_overall_rate})

school_type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
